In [2]:


import dotenv
from langgraph.prebuilt import ToolNode
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
dotenv.load_dotenv()

True

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import asyncio
async def mcp_tools_node():

    client = MultiServerMCPClient({
        "agent": {
            "transport": "streamable_http", # 
            "url": "http://localhost:2024/mcp", # 
        }
    })
    tools = await client.get_tools() # 

    return tools


In [4]:
tools = await mcp_tools_node()
print(tools)

[StructuredTool(name='relationship_agent', description='A chat agent that helps in solving relationship problems. it takes input in string formate in natural language and returns a response in string formate in natural language.', args_schema={'properties': {'question': {'title': 'Question', 'type': 'string'}}, 'required': ['question'], 'title': 'InputState', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x11a9fdee0>)]


In [5]:
tools[0].name, tools[0].description

('relationship_agent',
 'A chat agent that helps in solving relationship problems. it takes input in string formate in natural language and returns a response in string formate in natural language.')

In [6]:
#fix the description of the tool in .venv/lib/langgraph_api/api/mcp.py
# tools.append(
#     {
#         "name": name,
#         "inputSchema": schemas.get("input_schema", {}),
#         "description": assistant.get("description", ""), #<- this should be updated like this
#     },
# )

In [26]:
tools[0].name, tools[0].description

('relationship_agent',
 'A chat agent that helps in solving relationship problems. it takes input in string formate in natural language and returns a response in string formate in natural language.')

In [29]:
tool_node = ToolNode(tools)
AI_message = AIMessage(
            content="",
            tool_calls=[
                {
                    "name": "relationship_agent",
                    "args": {"question": "can you tell me how to make my partner happy?"},
                    "id": "tool_call_id_1",
                    "type": "tool_call_1",
                }
            ]
            
        )

result = await tool_node.ainvoke({"messages": [AI_message]})
print(result["messages"][-1].content)

{'answer': 'This is a wonderful question to be asking, as it shows you\'re invested in your partner\'s well-being and the health of your relationship. True happiness in a partnership isn\'t about grand gestures every day, but rather a consistent tapestry of understanding, respect, and thoughtful actions.\n\nHere\'s a comprehensive guide on how to make your partner happy, from a relationship expert\'s perspective:\n\n## The Golden Rule: Know Your Partner\n\nBefore diving into specific actions, the most crucial step is to **truly know your partner.** What makes *them* tick? What are *their* specific needs, desires, fears, and dreams? What are *their* "love languages"? (If you\'re unfamiliar, look up Gary Chapman\'s 5 Love Languages – Words of Affirmation, Quality Time, Receiving Gifts, Acts of Service, Physical Touch – it\'s incredibly insightful for understanding how people prefer to give and receive love.)\n\nOnce you have that foundational understanding, the following strategies becom

In [9]:
from src.agent.general_toolcalling_agent import GeneralAgentGraph
from langchain_core.messages import HumanMessage
agent_graph = GeneralAgentGraph(tools)
general_agent = agent_graph.get_graph()



In [12]:
result = general_agent.invoke({"messages": [HumanMessage(content="do you have access to any expert tool?")]})


In [17]:
result["messages"][-1].pretty_print()

================================== Ai Message ==================================

Yes, I have access to a relationship agent tool. It can help in solving relationship problems.


In [44]:
result = await general_agent.ainvoke({"messages": [HumanMessage(content="How should I make my girlfriend happy as I do not get time for her?")]})


In [45]:
for res in result["messages"]:
    res.pretty_print()

================================ Human Message =================================

How should I make my girlfriend happy as I do not get time for her?
================================== Ai Message ==================================
Tool Calls:
  relationship_agent (8f7710c7-889a-4add-8604-00341149d9e1)
 Call ID: 8f7710c7-889a-4add-8604-00341149d9e1
  Args:
    question: How should I make my girlfriend happy as I do not get time for her?
================================= Tool Message =================================
Name: relationship_agent

{'answer': 'It\'s incredibly thoughtful of you to be asking this question. Many people struggle with balancing demanding schedules and nurturing their relationships. The good news is that making your girlfriend feel loved and happy isn\'t always about the *quantity* of time, but the *quality* and *intentionality* of your efforts.\n\nHere\'s how you can make your girlfriend happy, even when time is scarce:\n\n**1. Communicate Openly and Honestly:**\n

In [46]:
general_agent.invoke({"messages": [HumanMessage(content="How should I make my girlfriend happy as I do not get time for her?")]})


{'messages': [HumanMessage(content='How should I make my girlfriend happy as I do not get time for her?', additional_kwargs={}, response_metadata={}, id='778a93d4-822a-47e6-9bfc-1cdbba2db3ea'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'relationship_agent', 'arguments': '{"question": "How should I make my girlfriend happy as I do not get time for her?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--c1d7c38b-117a-44a4-89b0-23622e0ae401-0', tool_calls=[{'name': 'relationship_agent', 'args': {'question': 'How should I make my girlfriend happy as I do not get time for her?'}, 'id': 'd9bb6623-317f-44f1-80f5-c9be61e425f2', 'type': 'tool_call'}], usage_metadata={'input_tokens': 53, 'output_tokens': 20, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content="Error: NotImplementedError('StructuredTool does not

In [41]:
# from langchain_core.messages import BaseMessage

# for chunk in general_agent.stream(
#     {"messages": [HumanMessage(content="How should I make my girlfriend happy as I do not get time for her?")]},
#     stream_mode="updates"
# ):
#     for step_name, step_output in chunk.items():
#         messages = step_output.get("messages")
#         if isinstance(messages, list):
#             for message in messages:
#                 if isinstance(message, BaseMessage):
#                     message.pretty_print()
#                 else:
#                     print(f"[{step_name}] {message}")  # fallback for ToolMessage or error strings
#         else:
#             # If it's not a list (e.g., a single AIMessage)
#             if isinstance(step_output, dict):
#                 message = step_output.get("messages")
#                 if isinstance(message, BaseMessage):
#                     message.pretty_print()
